In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [2]:
from pynq import Overlay
overlay = Overlay('./bitstream/LUT28bits.bit')

In [3]:
overlay.ip_dict

{'LUT28bits_0': {'addr_range': 65536,
  'device': <pynq.pl_server.device.XlnkDevice at 0xaf7f6690>,
  'driver': pynq.overlay.DefaultIP,
  'fullpath': 'LUT28bits_0',
  'gpio': {},
  'interrupts': {},
  'mem_id': 'S00_AXI',
  'parameters': {'C_S00_AXI_ADDR_WIDTH': '4',
   'C_S00_AXI_BASEADDR': '0x43C00000',
   'C_S00_AXI_DATA_WIDTH': '32',
   'C_S00_AXI_HIGHADDR': '0x43C0FFFF',
   'Component_Name': 'LUT28bits_LUT28bits_0_0',
   'EDK_IPTYPE': 'PERIPHERAL'},
  'phys_addr': 1136656384,
  'registers': {},
  'state': None,
  'type': 'xilinx.com:user:LUT28bits:1.0'}}

In [4]:
from pynq import MMIO
ip = MMIO(0x43C00000, 0x10000)

In [5]:
ip.device

### 將 r_LUT  的location輸出值轉為2補數:

In [6]:
def twos_complement_32bit(x):
    """將 32-bit 補數整數轉為有號值"""
    if x >= (1 << 31):
        return x - (1 << 32)
    else:
        return x

#### Test:

In [7]:
location_1 = -22 
remainder_1 = 11710

ip.write_mm(0, location_1)
ip.write_mm(8, remainder_1)


remainder = ip.read_mm(4)
location = ip.read_mm(12)


print(remainder)
print(twos_complement_32bit(location))

17128
-40


### 開始則中查表驗證

## 折衷查表:
### 用 Single Error Correction 解 Double Error Correction (A 取Double_AWE的 A)

#### 1.取 A的bits $\approx$ N的bits $\Rightarrow$ A = 18613  (30bits), N的範圍: 0 ~ 1073741823 (30bits)

### Define function
* (Not including 2's complement range

In [9]:
import math
import numpy as np


def decimal_to_binary(decimal_number):
    if decimal_number == 0:
        return "0"
    # 處理正數：直接轉換
    if decimal_number > 0:
        return bin(decimal_number)[2:]  # 移除 "0b" 前綴
       # 處理負數：自動計算所需位元數（2's complement）
    abs_value = abs(decimal_number)
    bits = abs_value.bit_length() + 1  # 負數多一個符號位元
    two_complement_value = (1 << bits) + decimal_number
    
    return bin(two_complement_value)[2:]  # 移除 "0b" 前綴

def bits_needed(number):
    return number.bit_length()


def bits_range(bits, signed=False):     ## 可選擇設定 有號數 或 無號數
    if bits <= 0:
        return "位元數必須為正整數！"
    
    if signed:
        # 二補數範圍: -2^(bits-1) 到 2^(bits-1)-1
        min_value = -(2**(bits - 1))
        max_value = 2**(bits - 1) - 1
    else:
        # 無符號範圍: 0 到 2^bits - 1
        min_value = 0
        max_value = 2**bits - 1

    return min_value, max_value

## 把重複的 AWE 去除
def remove_duplicates(input_list):
    unique_list = []
    for number in input_list:
        if number not in unique_list:
            unique_list.append(number)
    return unique_list


def get_sign(number):
    return 1 if number > 0 else -1 if number < 0 else 0


print("一個2補數所需要的位元數",bits_needed(120))
print("一bits數可表示的2補數的範圍",bits_range(bits_needed(120)))
print("一個2補數表達為二進制為:",decimal_to_binary(-8))

一個2補數所需要的位元數 7
一bits數可表示的2補數的範圍 (0, 127)
一個2補數表達為二進制為: 11000


In [10]:
### 輸入數值為data, ANcode的A為AN
### 乘完A後的數值為 input
data1 = (1 << 28) - 1    ## 
data2 = (1 << 28) - 19
data3 = (1 << 28) - 15
AN = 17619
input1 = data1*AN
input2 = data2*AN
input3 = data3*AN
bits1 = bits_needed(input1)
bits2 = bits_needed(input2)
bits3 = bits_needed(input3)
print("inputs1的位元數:", bits1)
print("inputs2的位元數:", bits2)
print("inputs3的位元數:", bits3)

inputs1的位元數: 43
inputs2的位元數: 43
inputs3的位元數: 43


### 產生1-bit AWE (Arithmetic-Weight Error)
* 並產生 Delta Q 的 list

In [11]:
AWE1_1=[]
e1_1=0     ## 錯誤數量
e1_data1=[]
h = [];   #位置
for i in range(0, bits1):
    e1_data1.insert(e1_1 , input1 + 2**i)
    AWE1_1.insert(e1_1, 2**i)
    e1_1 = e1_1 + 1
    e1_data1.insert(e1_1 , input1 - 2**i)
    AWE1_1.insert(e1_1, -2**i)
    e1_1 = e1_1 + 1
    h.insert(i,i+1)

AWE1_2=[]
e1_2=0     ## 錯誤數量
e1_data2=[]
for i in range(0, bits2):
    e1_data2.insert(e1_2 , input2 + 2**i)
    AWE1_2.insert(e1_2, 2**i)
    e1_2 = e1_2 + 1
    e1_data2.insert(e1_2 , input2 - 2**i)
    AWE1_2.insert(e1_2, -2**i)
    e1_2 = e1_2 + 1
    h.insert(i,i+1)
    
AWE1_3=[]
e1_3=0     ## 錯誤數量
e1_data3=[]
for i in range(0, bits3):
    e1_data3.insert(e1_3 , input3 + 2**i)
    AWE1_3.insert(e1_3, 2**i)
    e1_3 = e1_3 + 1
    e1_data3.insert(e1_3 , input3 - 2**i)
    AWE1_3.insert(e1_3, -2**i)
    e1_3 = e1_3 + 1
    h.insert(i,i+1)

h_awe = [val for num in h for val in (num, -num)]
print("產生1-bit AWE:", AWE1_1)
print("不同輸入資料的AWE是否相等:",AWE1_1 == AWE1_2 == AWE1_3)
print(h_awe)
print(np.size(AWE1_1))

產生1-bit AWE: [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104]
不同輸入資料的AWE是否相等: True
[1, -1, 2, -2, 3, -3, 4, -4, 5, -5, 6, -6, 7, -7, 8, -8, 9, -9, 10, -10, 11, -11, 12, -12, 13, -13, 14, -14, 15, -15, 16, -16, 17, -17, 18, -18, 19, -19, 20, -2

### 產生SEC的模數:

In [12]:
SEC_R = []
AWE1_R= []
for m in range(0,e1_1):
    mod = e1_data1[m] % AN
    SEC_R.append(mod)

for n in range(0,e1_1):
    mod_awe = AWE1_3[n] % AN
    AWE1_R.append(mod_awe)


print("SEC的模數:\n",SEC_R)
print("AWE1的模數:\n",SEC_R)
print("AWE1的模數是否等於SEC模數:", SEC_R == AWE1_R)
print("SEC的大小:",np.size(SEC_R))

SEC的模數:
 [1, 17618, 2, 17617, 4, 17615, 8, 17611, 16, 17603, 32, 17587, 64, 17555, 128, 17491, 256, 17363, 512, 17107, 1024, 16595, 2048, 15571, 4096, 13523, 8192, 9427, 16384, 1235, 15149, 2470, 12679, 4940, 7739, 9880, 15478, 2141, 13337, 4282, 9055, 8564, 491, 17128, 982, 16637, 1964, 15655, 3928, 13691, 7856, 9763, 15712, 1907, 13805, 3814, 9991, 7628, 2363, 15256, 4726, 12893, 9452, 8167, 1285, 16334, 2570, 15049, 5140, 12479, 10280, 7339, 2941, 14678, 5882, 11737, 11764, 5855, 5909, 11710, 11818, 5801, 6017, 11602, 12034, 5585]
AWE1的模數:
 [1, 17618, 2, 17617, 4, 17615, 8, 17611, 16, 17603, 32, 17587, 64, 17555, 128, 17491, 256, 17363, 512, 17107, 1024, 16595, 2048, 15571, 4096, 13523, 8192, 9427, 16384, 1235, 15149, 2470, 12679, 4940, 7739, 9880, 15478, 2141, 13337, 4282, 9055, 8564, 491, 17128, 982, 16637, 1964, 15655, 3928, 13691, 7856, 9763, 15712, 1907, 13805, 3814, 9991, 7628, 2363, 15256, 4726, 12893, 9452, 8167, 1285, 16334, 2570, 15049, 5140, 12479, 10280, 7339, 2941, 1467

### 產生2-bit AWE (Arithmetic-Weight Error)

In [13]:
AWE_2=[]
e2=0     ## 錯誤數量
p=0      ## 錯誤位置
e2_data=[]
for i in range(0, bits1):
    for j in range(i+1, bits1):
        e2_data.insert(e2 , (input1 + 2**i + 2**j))
        AWE_2.insert(e2 , (2**i + 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 + 2**i - 2**j))
        AWE_2.insert(e2 , (2**i - 2**j))        
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i + 2**j))
        AWE_2.insert(e2 , (-2**i + 2**j))                
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        e2_data.insert(e2 , (input1 - 2**i -2**j))
        AWE_2.insert(e2 , (-2**i - 2**j))
        print(f"第 {e2} 筆錯誤資料: {e2_data[e2]}")
        print(f"第 {e2} 筆算術錯誤: {AWE_2[e2]}")
        e2 = e2 + 1
        p = p + 1


print(e2)
print(p)
print("產生2-bits AWE:",AWE_2)
print("產生2-bits 錯誤資料:",e2_data)

第 0 筆錯誤資料: 4729564281648
第 0 筆算術錯誤: 3
第 1 筆錯誤資料: 4729564281644
第 1 筆算術錯誤: -1
第 2 筆錯誤資料: 4729564281646
第 2 筆算術錯誤: 1
第 3 筆錯誤資料: 4729564281642
第 3 筆算術錯誤: -3
第 4 筆錯誤資料: 4729564281650
第 4 筆算術錯誤: 5
第 5 筆錯誤資料: 4729564281642
第 5 筆算術錯誤: -3
第 6 筆錯誤資料: 4729564281648
第 6 筆算術錯誤: 3
第 7 筆錯誤資料: 4729564281640
第 7 筆算術錯誤: -5
第 8 筆錯誤資料: 4729564281654
第 8 筆算術錯誤: 9
第 9 筆錯誤資料: 4729564281638
第 9 筆算術錯誤: -7
第 10 筆錯誤資料: 4729564281652
第 10 筆算術錯誤: 7
第 11 筆錯誤資料: 4729564281636
第 11 筆算術錯誤: -9
第 12 筆錯誤資料: 4729564281662
第 12 筆算術錯誤: 17
第 13 筆錯誤資料: 4729564281630
第 13 筆算術錯誤: -15
第 14 筆錯誤資料: 4729564281660
第 14 筆算術錯誤: 15
第 15 筆錯誤資料: 4729564281628
第 15 筆算術錯誤: -17
第 16 筆錯誤資料: 4729564281678
第 16 筆算術錯誤: 33
第 17 筆錯誤資料: 4729564281614
第 17 筆算術錯誤: -31
第 18 筆錯誤資料: 4729564281676
第 18 筆算術錯誤: 31
第 19 筆錯誤資料: 4729564281612
第 19 筆算術錯誤: -33
第 20 筆錯誤資料: 4729564281710
第 20 筆算術錯誤: 65
第 21 筆錯誤資料: 4729564281582
第 21 筆算術錯誤: -63
第 22 筆錯誤資料: 4729564281708
第 22 筆算術錯誤: 63
第 23 筆錯誤資料: 4729564281580
第 23 筆算術錯誤: -65
第 24 筆錯誤資料: 4729564281774
第 24 筆算術錯誤

第 349 筆算術錯誤: -124
第 350 筆錯誤資料: 4729564281769
第 350 筆算術錯誤: 124
第 351 筆錯誤資料: 4729564281513
第 351 筆算術錯誤: -132
第 352 筆錯誤資料: 4729564281905
第 352 筆算術錯誤: 260
第 353 筆錯誤資料: 4729564281393
第 353 筆算術錯誤: -252
第 354 筆錯誤資料: 4729564281897
第 354 筆算術錯誤: 252
第 355 筆錯誤資料: 4729564281385
第 355 筆算術錯誤: -260
第 356 筆錯誤資料: 4729564282161
第 356 筆算術錯誤: 516
第 357 筆錯誤資料: 4729564281137
第 357 筆算術錯誤: -508
第 358 筆錯誤資料: 4729564282153
第 358 筆算術錯誤: 508
第 359 筆錯誤資料: 4729564281129
第 359 筆算術錯誤: -516
第 360 筆錯誤資料: 4729564282673
第 360 筆算術錯誤: 1028
第 361 筆錯誤資料: 4729564280625
第 361 筆算術錯誤: -1020
第 362 筆錯誤資料: 4729564282665
第 362 筆算術錯誤: 1020
第 363 筆錯誤資料: 4729564280617
第 363 筆算術錯誤: -1028
第 364 筆錯誤資料: 4729564283697
第 364 筆算術錯誤: 2052
第 365 筆錯誤資料: 4729564279601
第 365 筆算術錯誤: -2044
第 366 筆錯誤資料: 4729564283689
第 366 筆算術錯誤: 2044
第 367 筆錯誤資料: 4729564279593
第 367 筆算術錯誤: -2052
第 368 筆錯誤資料: 4729564285745
第 368 筆算術錯誤: 4100
第 369 筆錯誤資料: 4729564277553
第 369 筆算術錯誤: -4092
第 370 筆錯誤資料: 4729564285737
第 370 筆算術錯誤: 4092
第 371 筆錯誤資料: 4729564277545
第 371 筆算術錯

第 689 筆錯誤資料: 4729564248893
第 689 筆算術錯誤: -32752
第 690 筆錯誤資料: 4729564314397
第 690 筆算術錯誤: 32752
第 691 筆錯誤資料: 4729564248861
第 691 筆算術錯誤: -32784
第 692 筆錯誤資料: 4729564347197
第 692 筆算術錯誤: 65552
第 693 筆錯誤資料: 4729564216125
第 693 筆算術錯誤: -65520
第 694 筆錯誤資料: 4729564347165
第 694 筆算術錯誤: 65520
第 695 筆錯誤資料: 4729564216093
第 695 筆算術錯誤: -65552
第 696 筆錯誤資料: 4729564412733
第 696 筆算術錯誤: 131088
第 697 筆錯誤資料: 4729564150589
第 697 筆算術錯誤: -131056
第 698 筆錯誤資料: 4729564412701
第 698 筆算術錯誤: 131056
第 699 筆錯誤資料: 4729564150557
第 699 筆算術錯誤: -131088
第 700 筆錯誤資料: 4729564543805
第 700 筆算術錯誤: 262160
第 701 筆錯誤資料: 4729564019517
第 701 筆算術錯誤: -262128
第 702 筆錯誤資料: 4729564543773
第 702 筆算術錯誤: 262128
第 703 筆錯誤資料: 4729564019485
第 703 筆算術錯誤: -262160
第 704 筆錯誤資料: 4729564805949
第 704 筆算術錯誤: 524304
第 705 筆錯誤資料: 4729563757373
第 705 筆算術錯誤: -524272
第 706 筆錯誤資料: 4729564805917
第 706 筆算術錯誤: 524272
第 707 筆錯誤資料: 4729563757341
第 707 筆算術錯誤: -524304
第 708 筆錯誤資料: 4729565330237
第 708 筆算術錯誤: 1048592
第 709 筆錯誤資料: 4729563233085
第 709 筆算術錯誤: -1048560
第 710 筆

第 938 筆算術錯誤: 1099511627744
第 939 筆錯誤資料: 3630052653837
第 939 筆算術錯誤: -1099511627808
第 940 筆錯誤資料: 6928587537229
第 940 筆算術錯誤: 2199023255584
第 941 筆錯誤資料: 2530541026125
第 941 筆算術錯誤: -2199023255520
第 942 筆錯誤資料: 6928587537165
第 942 筆算術錯誤: 2199023255520
第 943 筆錯誤資料: 2530541026061
第 943 筆算術錯誤: -2199023255584
第 944 筆錯誤資料: 9127610792781
第 944 筆算術錯誤: 4398046511136
第 945 筆錯誤資料: 331517770573
第 945 筆算術錯誤: -4398046511072
第 946 筆錯誤資料: 9127610792717
第 946 筆算術錯誤: 4398046511072
第 947 筆錯誤資料: 331517770509
第 947 筆算術錯誤: -4398046511136
第 948 筆錯誤資料: 4729564281837
第 948 筆算術錯誤: 192
第 949 筆錯誤資料: 4729564281581
第 949 筆算術錯誤: -64
第 950 筆錯誤資料: 4729564281709
第 950 筆算術錯誤: 64
第 951 筆錯誤資料: 4729564281453
第 951 筆算術錯誤: -192
第 952 筆錯誤資料: 4729564281965
第 952 筆算術錯誤: 320
第 953 筆錯誤資料: 4729564281453
第 953 筆算術錯誤: -192
第 954 筆錯誤資料: 4729564281837
第 954 筆算術錯誤: 192
第 955 筆錯誤資料: 4729564281325
第 955 筆算術錯誤: -320
第 956 筆錯誤資料: 4729564282221
第 956 筆算術錯誤: 576
第 957 筆錯誤資料: 4729564281197
第 957 筆算術錯誤: -448
第 958 筆錯誤資料: 4729564282093
第 958 筆算術錯誤: 4

第 1188 筆算術錯誤: 4294967424
第 1189 筆錯誤資料: 4725269314477
第 1189 筆算術錯誤: -4294967168
第 1190 筆錯誤資料: 4733859248813
第 1190 筆算術錯誤: 4294967168
第 1191 筆錯誤資料: 4725269314221
第 1191 筆算術錯誤: -4294967424
第 1192 筆錯誤資料: 4738154216365
第 1192 筆算術錯誤: 8589934720
第 1193 筆錯誤資料: 4720974347181
第 1193 筆算術錯誤: -8589934464
第 1194 筆錯誤資料: 4738154216109
第 1194 筆算術錯誤: 8589934464
第 1195 筆錯誤資料: 4720974346925
第 1195 筆算術錯誤: -8589934720
第 1196 筆錯誤資料: 4746744150957
第 1196 筆算術錯誤: 17179869312
第 1197 筆錯誤資料: 4712384412589
第 1197 筆算術錯誤: -17179869056
第 1198 筆錯誤資料: 4746744150701
第 1198 筆算術錯誤: 17179869056
第 1199 筆錯誤資料: 4712384412333
第 1199 筆算術錯誤: -17179869312
第 1200 筆錯誤資料: 4763924020141
第 1200 筆算術錯誤: 34359738496
第 1201 筆錯誤資料: 4695204543405
第 1201 筆算術錯誤: -34359738240
第 1202 筆錯誤資料: 4763924019885
第 1202 筆算術錯誤: 34359738240
第 1203 筆錯誤資料: 4695204543149
第 1203 筆算術錯誤: -34359738496
第 1204 筆錯誤資料: 4798283758509
第 1204 筆算術錯誤: 68719476864
第 1205 筆錯誤資料: 4660844805037
第 1205 筆算術錯誤: -68719476608
第 1206 筆錯誤資料: 4798283758253
第 1206 筆算術錯誤: 68719476608
第

...

第 2437 筆錯誤資料: 4729530989357
第 2437 筆算術錯誤: -33292288
第 2438 筆錯誤資料: 4729597573933
第 2438 筆算術錯誤: 33292288
第 2439 筆錯誤資料: 4729530465069
第 2439 筆算術錯誤: -33816576
第 2440 筆錯誤資料: 4729631652653
第 2440 筆算術錯誤: 67371008
第 2441 筆錯誤資料: 4729497434925
第 2441 筆算術錯誤: -66846720
第 2442 筆錯誤資料: 4729631128365
第 2442 筆算術錯誤: 66846720
第 2443 筆錯誤資料: 4729496910637
第 2443 筆算術錯誤: -67371008
第 2444 筆錯誤資料: 4729698761517
第 2444 筆算術錯誤: 134479872
第 2445 筆錯誤資料: 4729430326061
第 2445 筆算術錯誤: -133955584
第 2446 筆錯誤資料: 4729698237229
第 2446 筆算術錯誤: 133955584
第 2447 筆錯誤資料: 4729429801773
第 2447 筆算術錯誤: -134479872
第 2448 筆錯誤資料: 4729832979245
第 2448 筆算術錯誤: 268697600
第 2449 筆錯誤資料: 4729296108333
第 2449 筆算術錯誤: -268173312
第 2450 筆錯誤資料: 4729832454957
第 2450 筆算術錯誤: 268173312
第 2451 筆錯誤資料: 4729295584045
第 2451 筆算術錯誤: -268697600
第 2452 筆錯誤資料: 4730101414701
第 2452 筆算術錯誤: 537133056
第 2453 筆錯誤資料: 4729027672877
第 2453 筆算術錯誤: -536608768
第 2454 筆錯誤資料: 4730100890413
第 2454 筆算術錯誤: 536608768
第 2455 筆錯誤資料: 4729027148589
第 2455 筆算術錯誤: -537133056
第 2456 筆錯

第 2687 筆錯誤資料: 331516721965
第 2687 筆算術錯誤: -4398047559680
第 2688 筆錯誤資料: 4729570573101
第 2688 筆算術錯誤: 6291456
第 2689 筆錯誤資料: 4729562184493
第 2689 筆算術錯誤: -2097152
第 2690 筆錯誤資料: 4729566378797
第 2690 筆算術錯誤: 2097152
第 2691 筆錯誤資料: 4729557990189
第 2691 筆算術錯誤: -6291456
第 2692 筆錯誤資料: 4729574767405
第 2692 筆算術錯誤: 10485760
第 2693 筆錯誤資料: 4729557990189
第 2693 筆算術錯誤: -6291456
第 2694 筆錯誤資料: 4729570573101
第 2694 筆算術錯誤: 6291456
第 2695 筆錯誤資料: 4729553795885
第 2695 筆算術錯誤: -10485760
第 2696 筆錯誤資料: 4729583156013
第 2696 筆算術錯誤: 18874368
第 2697 筆錯誤資料: 4729549601581
第 2697 筆算術錯誤: -14680064
第 2698 筆錯誤資料: 4729578961709
第 2698 筆算術錯誤: 14680064
第 2699 筆錯誤資料: 4729545407277
第 2699 筆算術錯誤: -18874368
第 2700 筆錯誤資料: 4729599933229
第 2700 筆算術錯誤: 35651584
第 2701 筆錯誤資料: 4729532824365
第 2701 筆算術錯誤: -31457280
第 2702 筆錯誤資料: 4729595738925
第 2702 筆算術錯誤: 31457280
第 2703 筆錯誤資料: 4729528630061
第 2703 筆算術錯誤: -35651584
第 2704 筆錯誤資料: 4729633487661
第 2704 筆算術錯誤: 69206016
第 2705 筆錯誤資料: 4729499269933
第 2705 筆算術錯誤: -65011712
第 2706 筆錯誤資料: 472962929

第 2936 筆算術錯誤: 150994944
第 2937 筆錯誤資料: 4729446841133
第 2937 筆算術錯誤: -117440512
第 2938 筆錯誤資料: 4729681722157
第 2938 筆算術錯誤: 117440512
第 2939 筆錯誤資料: 4729413286701
第 2939 筆算術錯誤: -150994944
第 2940 筆錯誤資料: 4729849494317
第 2940 筆算術錯誤: 285212672
第 2941 筆錯誤資料: 4729312623405
第 2941 筆算術錯誤: -251658240
第 2942 筆錯誤資料: 4729815939885
第 2942 筆算術錯誤: 251658240
第 2943 筆錯誤資料: 4729279068973
第 2943 筆算術錯誤: -285212672
第 2944 筆錯誤資料: 4730117929773
第 2944 筆算術錯誤: 553648128
第 2945 筆錯誤資料: 4729044187949
第 2945 筆算術錯誤: -520093696
第 2946 筆錯誤資料: 4730084375341
第 2946 筆算術錯誤: 520093696
第 2947 筆錯誤資料: 4729010633517
第 2947 筆算術錯誤: -553648128
第 2948 筆錯誤資料: 4730654800685
第 2948 筆算術錯誤: 1090519040
第 2949 筆錯誤資料: 4728507317037
第 2949 筆算術錯誤: -1056964608
第 2950 筆錯誤資料: 4730621246253
第 2950 筆算術錯誤: 1056964608
第 2951 筆錯誤資料: 4728473762605
第 2951 筆算術錯誤: -1090519040
第 2952 筆錯誤資料: 4731728542509
第 2952 筆算術錯誤: 2164260864
第 2953 筆錯誤資料: 4727433575213
第 2953 筆算術錯誤: -2130706432
第 2954 筆錯誤資料: 4731694988077
第 2954 筆算術錯誤: 2130706432
第 2955 筆錯誤資料: 4727400020

第 3186 筆算術錯誤: 2198889037824
第 3187 筆錯誤資料: 2530406808365
第 3187 筆算術錯誤: -2199157473280
第 3188 筆錯誤資料: 9127745010477
第 3188 筆算術錯誤: 4398180728832
第 3189 筆錯誤資料: 331651988269
第 3189 筆算術錯誤: -4397912293376
第 3190 筆錯誤資料: 9127476575021
第 3190 筆算術錯誤: 4397912293376
第 3191 筆錯誤資料: 331383552813
第 3191 筆算術錯誤: -4398180728832
第 3192 筆錯誤資料: 4730369588013
第 3192 筆算術錯誤: 805306368
第 3193 筆錯誤資料: 4729295846189
第 3193 筆算術錯誤: -268435456
第 3194 筆錯誤資料: 4729832717101
第 3194 筆算術錯誤: 268435456
第 3195 筆錯誤資料: 4728758975277
第 3195 筆算術錯誤: -805306368
第 3196 筆錯誤資料: 4730906458925
第 3196 筆算術錯誤: 1342177280
第 3197 筆錯誤資料: 4728758975277
第 3197 筆算術錯誤: -805306368
第 3198 筆錯誤資料: 4730369588013
第 3198 筆算術錯誤: 805306368
第 3199 筆錯誤資料: 4728222104365
第 3199 筆算術錯誤: -1342177280
第 3200 筆錯誤資料: 4731980200749
第 3200 筆算術錯誤: 2415919104
第 3201 筆錯誤資料: 4727685233453
第 3201 筆算術錯誤: -1879048192
第 3202 筆錯誤資料: 4731443329837
第 3202 筆算術錯誤: 1879048192
第 3203 筆錯誤資料: 4727148362541
第 3203 筆算術錯誤: -2415919104
第 3204 筆錯誤資料: 4734127684397
第 3204 筆算術錯誤: 4563402752
第 

第 3436 筆錯誤資料: 4772513954605
第 3436 筆算術錯誤: 42949672960
第 3437 筆錯誤資料: 4703794477869
第 3437 筆算術錯誤: -25769803776
第 3438 筆錯誤資料: 4755334085421
第 3438 筆算術錯誤: 25769803776
第 3439 筆錯誤資料: 4686614608685
第 3439 筆算術錯誤: -42949672960
第 3440 筆錯誤資料: 4806873692973
第 3440 筆算術錯誤: 77309411328
第 3441 筆錯誤資料: 4669434739501
第 3441 筆算術錯誤: -60129542144
第 3442 筆錯誤資料: 4789693823789
第 3442 筆算術錯誤: 60129542144
第 3443 筆錯誤資料: 4652254870317
第 3443 筆算術錯誤: -77309411328
第 3444 筆錯誤資料: 4875593169709
第 3444 筆算術錯誤: 146028888064
第 3445 筆錯誤資料: 4600715262765
第 3445 筆算術錯誤: -128849018880
第 3446 筆錯誤資料: 4858413300525
第 3446 筆算術錯誤: 128849018880
第 3447 筆錯誤資料: 4583535393581
第 3447 筆算術錯誤: -146028888064
第 3448 筆錯誤資料: 5013032123181
第 3448 筆算術錯誤: 283467841536
第 3449 筆錯誤資料: 4463276309293
第 3449 筆算術錯誤: -266287972352
第 3450 筆錯誤資料: 4995852253997
第 3450 筆算術錯誤: 266287972352
第 3451 筆錯誤資料: 4446096440109
第 3451 筆算術錯誤: -283467841536
第 3452 筆錯誤資料: 5287910030125
第 3452 筆算術錯誤: 558345748480
第 3453 筆錯誤資料: 4188398402349
第 3453 筆算術錯誤: -541165879296
第 3454 筆錯

In [14]:
DEC_R = []
for m in range(0,e2):
    mod = e2_data[m] % AN
    DEC_R.append(mod)

print("DEC_R的模數:\n",DEC_R)
print(np.size(DEC_R))

DEC_R的模數:
 [3, 17618, 1, 17616, 5, 17616, 3, 17614, 9, 17612, 7, 17610, 17, 17604, 15, 17602, 33, 17588, 31, 17586, 65, 17556, 63, 17554, 129, 17492, 127, 17490, 257, 17364, 255, 17362, 513, 17108, 511, 17106, 1025, 16596, 1023, 16594, 2049, 15572, 2047, 15570, 4097, 13524, 4095, 13522, 8193, 9428, 8191, 9426, 16385, 1236, 16383, 1234, 15150, 2471, 15148, 2469, 12680, 4941, 12678, 4939, 7740, 9881, 7738, 9879, 15479, 2142, 15477, 2140, 13338, 4283, 13336, 4281, 9056, 8565, 9054, 8563, 492, 17129, 490, 17127, 983, 16638, 981, 16636, 1965, 15656, 1963, 15654, 3929, 13692, 3927, 13690, 7857, 9764, 7855, 9762, 15713, 1908, 15711, 1906, 13806, 3815, 13804, 3813, 9992, 7629, 9990, 7627, 2364, 15257, 2362, 15255, 4727, 12894, 4725, 12892, 9453, 8168, 9451, 8166, 1286, 16335, 1284, 16333, 2571, 15050, 2569, 15048, 5141, 12480, 5139, 12478, 10281, 7340, 10279, 7338, 2942, 14679, 2940, 14677, 5883, 11738, 5881, 11736, 11765, 5856, 11763, 5854, 5910, 11711, 5908, 11709, 11819, 5802, 11817, 5800, 

In [15]:
total_AWE = AWE1_1 + AWE_2
e_total = e1_data1 + e2_data
R_total = SEC_R + DEC_R
print("還未去除重複error data的數量:",np.size(e_total))
print("還未去除重複餘數的數量:", np.size(R_total))

## 把重複的 AWE 及 error_data 去除
AWE = remove_duplicates(total_AWE)
new_e_total = remove_duplicates(e_total)
new_R_total = remove_duplicates(R_total)
error_total_count = np.size(new_e_total)
print("已去除重複error data的數量:",error_total_count)
print("已未去除重複餘數的數量:",np.size(new_R_total))
print("已未去除重複AWE的數量:",np.size(AWE))
print("所有的AWE為:\n",AWE)
print("所有算術錯誤的資料:\n", new_e_total)

還未去除重複error data的數量: 3698
還未去除重複餘數的數量: 3698
已去除重複error data的數量: 3532
已未去除重複餘數的數量: 3532
已未去除重複AWE的數量: 3532
所有的AWE為:
 [1, -1, 2, -2, 4, -4, 8, -8, 16, -16, 32, -32, 64, -64, 128, -128, 256, -256, 512, -512, 1024, -1024, 2048, -2048, 4096, -4096, 8192, -8192, 16384, -16384, 32768, -32768, 65536, -65536, 131072, -131072, 262144, -262144, 524288, -524288, 1048576, -1048576, 2097152, -2097152, 4194304, -4194304, 8388608, -8388608, 16777216, -16777216, 33554432, -33554432, 67108864, -67108864, 134217728, -134217728, 268435456, -268435456, 536870912, -536870912, 1073741824, -1073741824, 2147483648, -2147483648, 4294967296, -4294967296, 8589934592, -8589934592, 17179869184, -17179869184, 34359738368, -34359738368, 68719476736, -68719476736, 137438953472, -137438953472, 274877906944, -274877906944, 549755813888, -549755813888, 1099511627776, -1099511627776, 2199023255552, -2199023255552, 4398046511104, -4398046511104, 3, -3, 5, -5, 9, -7, 7, -9, 17, -15, 15, -17, 33, -31, 31, -33, 65, -63, 63, -

### 所有AWE數量: $4*C^{bits}_{2}$ $-$ $2*(bits-1)$ - $2*(bits-2)$
* $2*(bits-1)$: 所有的 single AWE都可以分解成已重複的 double AWE, 除了MSB例外
* EX: $(+,0)$ 只能分解成 $(0,++)$ , $(-,0)$ 只能分解成 $(0,--)$
* $2*(bits-2)$: double AWE重複的個數
* EX: $(+,0,-)$ = $(0,+,+)$ and $(-,0,+)$ = $(0,-,-)$

### Trade-off Algorithm: 2025/05/27 使用 Vivado HLX IP 驗證
* 此解出來的數字為位置及正負號 EX: $-1$ 代表 $-2^0$ , $-5$ 代表 $-2^4$ 以此類推

In [17]:
import math
import time
### 給定一個DEC_R, 找錯誤位置h1及h2(包含錯誤正負號)
import numpy as np

h1 = 0; h2 = 0; h_temp = 0
possible_h1 = []; possible_h2 = []; possible_AWE = []
output = []; possible_SE_h1=[]

start_time = time.time()  # 記錄開始時間
for R, error in zip(new_R_total, new_e_total): 
    found = False                                       # 旗標，用來標示是否要跳出 for H 循環 
    for H in range(0,bits1):                              
        for s in range(-1,2,2):                       
            h1 = s*(H+1)
            ip.write_mm(0, h1)
            R1 = ip.read_mm(4)
            R2 = R - R1
            if R2 < 0:
                R2 = AN + R2
            
            ip.write_mm(8, R2)
            h2 = twos_complement_32bit(ip.read_mm(12))                  
            if (h2 != 0):
                s2 = get_sign(h2)                     
                possible_h1.append(h1)                  # 若找不到對應的h2,h2定為0(函數內容已定義)
                possible_h2.append(h2)                  # 代表錯誤可能是三個或以上(multiple errors)
                output.append((error - (s*2**(abs(h1)-1)) - (s2 * 2**(abs(h2)-1))) // AN)
                found = True
                break
        if found:
            break
            
end_time = time.time()  # 記錄結束時間
elapsed_time = end_time - start_time  # 計算執行時間
print(f"執行時間: {elapsed_time:.6f} 秒")
print("更正後的值是否和原data相同:", data1 == output.pop())
print(elapsed_time / np.size(possible_h1))
##print(possible_h1)
##print(possible_h2)
##print("更正後的值:",output)
print(np.size(possible_h1))
print(np.size(possible_h2))
##print("輸入的 error data的總數是否和經過演算法後相同:", error_total_count == np.size(possible_h1))
##print(possible_SE_h1)

執行時間: 11.378128 秒
更正後的值是否和原data相同: True
0.0032214406257577554
3532
3532
